In [31]:
import torch
import pandas as pd
import numpy as np

from sklearn.base import ClassifierMixin, RegressorMixin
from tsml.base import BaseTimeSeriesEstimator

from models.random_feature_representation_boosting import HydraBoost


class TSMLWrapperHydraBoost(RegressorMixin, BaseTimeSeriesEstimator):
    
    def __init__(self, **kwargs):
        super(TSMLWrapperHydraBoost, self).__init__()
        self.hydraboost = HydraBoost(
            n_layers=1,
            init_n_kernels=8,
            init_n_groups=64,
            n_kernels=8,
            n_groups=64,
            max_num_channels=3,
            hydra_batch_size=10000,
            l2_reg=10,
            l2_ghat=0.1,
            boost_lr=1,
            train_top_at = [0, 5, 10],
            **kwargs
        )
        

    def fit(self, X: np.ndarray, y: np.ndarray) -> object:
        """Fit the estimator to training data.

        Parameters
        ----------
        X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
            The training data.
        y : 1D np.ndarray of shape (n_instances)
            The target labels for fitting, indices correspond to instance indices in X

        Returns
        -------
        self :
            Reference to self.
        """
        X = torch.from_numpy(X).float()
        y = torch.from_numpy(y).float()
        y = y.unsqueeze(1)
        self.X_mean = X.mean()
        self.X_std = X.std()
        self.y_mean = y.mean()
        self.y_std = y.std()
        X = (X - self.X_mean) / self.X_std
        y = (y - self.y_mean) / self.y_std
        self.hydraboost.fit(X, y)
        return self


    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predicts labels for sequences in X.

        Parameters
        ----------
        X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
            The training data.

        Returns
        -------
        y : array-like of shape (n_instances)
            Predicted target labels.
        """
        X = torch.from_numpy(X).float()
        X = (X - self.X_mean) / self.X_std
        pred = self.hydraboost(X)
        pred = pred * self.y_std + self.y_mean
        return pred.squeeze().detach().numpy()
        
        

    def _more_tags(self) -> dict:
        return {
            "X_types": ["3darray"],
            "equal_length_only": True,
            "allow_nan": False,
        }


In [32]:
# test hydra

from load_datasets import get_aeon_dataset
from pathlib import Path
import os
import torch
import numpy as np

current_dir = Path(os.path.dirname(os.getcwd()))
TSER_data_dir = current_dir.parent / "Data" / "TSER"

dataset_name = "HouseholdPowerConsumption1"
X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name, TSER_data_dir, "regression")

In [33]:
from tsml_eval.experiments import (
    experiments,
    get_regressor_by_name,
    run_regression_experiment,
)

regressor = TSMLWrapperHydraBoost()
regressor_name = "HydraBoost"
run_regression_experiment(
    X_train,
    y_train,
    X_test,
    y_test,
    regressor,
    regressor_name=regressor_name,
    results_path="results/",
    dataset_name=dataset_name,
    resample_id=0,
)


training W0
Phi0 shape torch.Size([745, 8192])


In [35]:
from tsml_eval.evaluation.storage import load_regressor_results

rr = load_regressor_results(
    current_dir / "exploring-hydra-boosting" /"results" / regressor_name / "Predictions" / dataset_name / "testResample0.csv"
)
print(rr.predictions)
print(rr.mean_squared_error, "mse")
print(rr.root_mean_squared_error, "rmse")
print(rr.mean_absolute_percentage_error, "mape")
print(rr.r2_score, "r2")


[1510.00549316 1170.70849609 1178.43505859 1622.09436035 1858.77246094
 1495.05932617 1628.91577148 1896.97802734 1650.92932129 2544.23486328
 1278.6529541  1892.38989258 1651.71606445 1585.17956543 1465.96191406
 1504.0057373  1239.76599121 1250.55957031 1211.29248047  808.8336792
  601.12329102  950.824646   1650.59765625 2180.95556641 2282.46972656
 1868.14770508 1752.98742676 1815.85534668 2140.64648438 1978.05517578
 2137.33740234 1913.60131836 1775.59741211 1826.87768555 1917.11694336
 1232.08532715 2106.20776367 1533.87634277 1655.32202148 1154.3416748
 1184.1081543  1064.35253906  988.13885498  647.23150635 1043.73535156
 1355.83752441 2259.45776367 1776.95117188 2069.63696289 1636.27404785
 2099.3996582  1776.62316895 1781.61425781 2733.54199219 1405.57336426
 2496.07421875 1976.60913086 2427.68579102 1650.45593262 1448.57824707
 1581.78271484 1203.49243164 1465.91894531 1122.64611816 1184.24475098
 1053.58325195  582.07012939 1254.50341797 1353.69897461 1698.59179688
 1850.66

c:\Users\nz423\Code\exploring-hydra-boosting\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
